# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../output_data/cities.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Latitude,Longitude
0,hithadhoo,81.82,74,100,6.89,MV,1604389711,-0.60,73.08
1,nhulunbuy,87.80,58,100,12.75,AU,1604389711,-12.23,136.77
2,barrow,28.40,79,90,23.04,US,1604389501,71.29,-156.79
3,punta arenas,48.09,76,75,6.93,CL,1604389606,-53.15,-70.92
4,busselton,66.20,38,65,14.12,AU,1604389712,-33.65,115.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
latLongDF = weather_df.loc[:, ["Latitude", "Longitude"]]
coordinatesDF = latLongDF.to_records(index=False)
coordinates = list(coordinatesDF)

In [5]:
figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
trimming_df = weather_df.loc[(weather_df["Temperature"] > 60) & (weather_df["Temperature"] < 75) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] < 20), :]
idealWeather_df = trimming_df.iloc[range(0,10),:]
idealWeather_df

,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Latitude,Longitude
31,guerrero negro,68.02,80,0,9.15,MX,1604389594,27.98,-114.06
74,cooma,71.60,37,0,5.82,AU,1604389728,-36.23,149.13
75,buenos aires,62.89,82,0,8.05,AR,1604389500,-34.61,-58.38
90,santiago del estero,62.40,46,0,1.95,AR,1604389732,-27.80,-64.26
98,navolato,70.03,81,2,3.00,MX,1604389733,24.77,-107.69
118,hermanus,73.99,69,4,5.01,ZA,1604389682,-34.42,19.23
130,tiznit,67.98,25,0,4.76,MA,1604389740,29.58,-9.50
135,flinders,64.99,71,0,3.00,AU,1604389741,-34.58,150.86
223,shouguang,62.01,21,0,1.01,CN,1604389762,36.88,118.74
264,hermosillo,73.11,34,0,3.00,MX,1604389478,29.07,-110.97


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in idealWeather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        idealWeather_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\Calvin Bauer\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Calvin Bauer\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


In [8]:
hotel_df = idealWeather_df
hotel_df.head(10)

,City,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Latitude,Longitude,Hotel Name
31,guerrero negro,68.02,80,0,9.15,MX,1604389594,27.98,-114.06,Hotel TerraSal
74,cooma,71.60,37,0,5.82,AU,1604389728,-36.23,149.13,Altair Motel Cooma
75,buenos aires,62.89,82,0,8.05,AR,1604389500,-34.61,-58.38,Hilton Buenos Aires
90,santiago del estero,62.40,46,0,1.95,AR,1604389732,-27.80,-64.26,Hotel Altos del Estero
98,navolato,70.03,81,2,3.00,MX,1604389733,24.77,-107.69,Hotel San Francisco
118,hermanus,73.99,69,4,5.01,ZA,1604389682,-34.42,19.23,Misty Waves Boutique Hotel
130,tiznit,67.98,25,0,4.76,MA,1604389740,29.58,-9.50,NaN
135,flinders,64.99,71,0,3.00,AU,1604389741,-34.58,150.86,Eternity 141
223,shouguang,62.01,21,0,1.01,CN,1604389762,36.88,118.74,Blue Horizon Hotel
264,hermosillo,73.11,34,0,3.00,MX,1604389478,29.07,-110.97,Marsella 45


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

In [12]:
info_box_content=hotel_info